In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import pytensor.tensor as at
from pytensor import scan
import pytensor.tensor as at
import pytensor.tensor.signal.conv as conv

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


In [2]:
rng = np.random.default_rng(42)

# ---- Setup ----
n_products = 5
n_weeks = 104  # 2 years of weekly data
weeks = np.arange(n_weeks)
products = np.arange(n_products)

# ---- Functions ----
def adstock(x: np.ndarray, lam: float) -> np.ndarray:
    """Simple adstock with decay lam in [0,1)."""
    if not 0 <= lam < 1:
        raise ValueError("lam must be in [0,1).")
    out = np.zeros_like(x, dtype=float)
    for t in range(len(x)):
        out[t] = x[t] + (out[t-1] * lam if t > 0 else 0.0)
    return out


def hill_saturation(x: np.ndarray, alpha: float, half_saturation: float) -> np.ndarray:
    """Hill/EC50-style saturation; positive, increasing, bounded."""
    if alpha <= 0 or half_saturation <= 0:
        raise ValueError("alpha and half_saturation must be > 0.")
    return (x**alpha) / (x**alpha + half_saturation**alpha + 1e-12)


# ---- Generate Panel Data ----
panel_data = []

for p in products:
    # Product-specific parameters
    # Incorporate probability distributions to assign different transformations to channels per product

    # TV
    alpha_tv = rng.normal(20000, 2000)
    theta_tv = rng.normal(5000, 500) 
    gamma_tv = rng.normal(1.5, 0.5)
    lam_tv = rng.normal(0.5, 0.2)
    
    # Search 
    alpha_search = rng.normal(30000, 2000)
    theta_search = rng.normal(10000, 500) 
    gamma_search = rng.normal(2, 1)
    lam_search = rng.normal(0.2, 0.2)
    
    # Social 
    alpha_social = rng.normal(10000, 5000)
    theta_social = rng.normal(5000, 500) 
    gamma_social = rng.normal(2, 1)
    lam_social = rng.normal(0.3, 0.1)
    
    
    # Media spend (random but with product-specific scale)
    spend_tv = rng.gamma(5, 1000, n_weeks) * (1 + 0.2 * p) # (1 + 0.2 * p) scales spend by products so high product get more spend
    spend_search = rng.gamma(4, 800, n_weeks) * (1 + 0.1 * p)
    spend_social = rng.gamma(3, 500, n_weeks) * (1 + 0.05 * p)

    # Apply adstock
    ad_tv = adstock_transform(spend_tv, lam_tv)
    ad_search = adstock_transform(spend_search, lam_search)
    ad_social = adstock_transform(spend_social, lam_social)

    # Apply saturation
    eff_tv = hill_transform(ad_tv, theta_tv, gamma_tv)
    eff_search = hill_transform(ad_search, theta_search, gamma_search)
    eff_social = hill_transform(ad_social, theta_social, gamma_social)

    # Controls
    price = rng.normal(100, 5, n_weeks) * (1 + 0.05 * p)
    gdp_index = 100 + 0.1 * weeks + rng.normal(0, 2, n_weeks)

    # Seasonality (yearly cycle)
    seasonality = 20000 * np.sin(2 * np.pi * weeks / 52) + 10000 * np.cos(2 * np.pi * weeks / 52)

    # Nonlinear trend (logistic growth baseline)
    baseline = 50000 / (1 + np.exp(-0.05 * (weeks - 50)))

    # Combine everything to create KPI (sales)
    kpi = (
        baseline
        + eff_tv + eff_search + eff_social
        - 200 * (price - 100)        # price effect (higher price → lower sales)
        + 100 * gdp_index            # macro control
        + seasonality
        + rng.normal(0, 5000, n_weeks)  # noise
    )

    df = pd.DataFrame({
        "product_id": p,
        "week": weeks,
        "spend_tv": spend_tv,
        "spend_search": spend_search,
        "spend_social": spend_social,
        "price": price,
        "gdp_index": gdp_index,
        "kpi_sales": kpi
    })
    panel_data.append(df)

panel = pd.concat(panel_data, ignore_index=True)

print(panel.head())

   product_id  week     spend_tv  spend_search  spend_social       price  \
0           0     0  4810.767563   2702.957448   1409.977856   97.941808   
1           0     1  5751.209146   2603.597554    904.511228  106.813213   
2           0     2  5509.444788   3212.222484    731.694374   94.797070   
3           0     3  6833.183279   3074.148164   2102.967174   87.936098   
4           0     4  4278.601371   8726.736533    493.246639  108.054685   

    gdp_index     kpi_sales  
0  100.895627  20530.283071  
1  100.216549  20497.935934  
2  101.297478  28194.390910  
3   99.924658  40339.402609  
4  100.956287  40378.226800  


C:\Users\JackGlover\AppData\Local\Temp\ipykernel_26308\3497069973.py:18: RuntimeWarning: invalid value encountered in power
  return (x**gamma) / (theta**gamma + x**gamma + 1e-8)
